In [9]:
import pypesto
import pypesto.petab
import pypesto.optimize as optimize
import pypesto.sample as sample
import pypesto.visualize as visualize

import petab
import numpy as np
from scipy.special import gammaln
from scipy.special import gamma
import pickle

datatype = "original"

# import to petab
if datatype == "original":
    petab_problem = petab.Problem.from_yaml(
    "corrupted_data/SS_conversion_reaction_original.yaml")
elif datatype == "switch":
    petab_problem = petab.Problem.from_yaml(
    "corrupted_data/SS_conversion_reaction_switch.yaml")
else:
    petab_problem = petab.Problem.from_yaml(
    "corrupted_data/SS_conversion_reaction_loss .yaml")

In [10]:
def analytical_b(t, a0, b0, k1, k2):
    return (k2 - k2 * np.exp(-(k2 + k1) * t)) / (k2 + k1)

def simulate_model(x, tvec):
    # assign parameters
    k1, k2, _ = x
    # define initial conditions
    a0 = 1
    b0 = 0
    # simulate model
    simulation = [analytical_b(t, a0, b0, k1, k2)
                   for t in tvec]
    return simulation

After importing the model, we need to define the objective function. This time we will do it via an external function that will be used then by pyPESTO instead of using the built-in ones.

For numerical reasons we will implement the log likelihood and log prior.

In [27]:
def negative_log_marginalised_likelihood(x):
    """ Negative log posterior function."""

    scale = x[2]

    # experimental data
    data = np.asarray(petab_problem.measurement_df.measurement)
    # time vector
    tvec = np.asarray(petab_problem.measurement_df.time)

    N = len(tvec)

    # simulate model
    _simulation = simulate_model(np.exp(x), tvec)
    simulation = np.asarray(_simulation)

    # evaluate standard log likelihood
    res = data - simulation
    b_vector = sorted(res)
    b_vector.append(0)
    #r is the value for which b_(r+1) is the first vale which ich >= 0. To access that value in b_vector however
    #one has to insert r-1
    r = next(x for x, val in enumerate(b_vector) if val >= 0)
    b_vector = b_vector[:-1]
    h_vector = np.zeros(N-r+1)
    l_vector = np.zeros(N-r+1)
    l_llh = np.zeros(N-r+1)
    
    
    h_vector[0] = (lamda / (2*(N - 2*r - lamda))) * (np.exp(b_vector[r] * (N - 2*r - lamda)) - 1)
    for i in range(1, N-r):
        const = N - 2*(i + r) - lamda
        h_vector[i] = (lamda / (2 * const)) * (np.exp(b_vector[i + r] * const) - np.exp(b_vector[i + r -1] * const))
    h_vector[N-r] = (lamda / (2 * (N + lamda))) * np.exp(b_vector[N-1] * (-N - lamda))
    
    if r == 0:
        l_vector[0] = - np.sum(b_vector)
    else:
        l_vector[0] = np.sum(b_vector[:r - 1]) - np.sum(b_vector[r:])
    
    print(l_vector[0])
    for i in range(1, N-r+1):
        l_vector[i] = l_vector[i - 1] + 2*b_vector[i + r - 1]
        print(l_vector)
    
    for i in range(N-r+1):
        l_llh[i] = (h_vector[i]/scale) * np.exp(l_vector[i]/scale)
    
    return -np.log(np.sum(l_llh))

x0 = np.array([-1.5, -0.5,  0.1])
z = negative_log_marginalised_likelihood(x0)

-11.236571144995112
[-11.23657114  -9.25249419   0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.        ]
[-11.23657114  -9.25249419  -7.2431228    0.           0.
   0.           0.           0.           0.           0.
   0.           0.        ]
[-11.23657114  -9.25249419  -7.2431228   -5.23307536   0.
   0.           0.           0.           0.           0.
   0.           0.        ]
[-11.23657114  -9.25249419  -7.2431228   -5.23307536  -3.20552136
   0.           0.           0.           0.           0.
   0.           0.        ]
[-11.23657114  -9.25249419  -7.2431228   -5.23307536  -3.20552136
  -1.16490707   0.           0.           0.           0.
   0.           0.        ]
[-11.23657114  -9.25249419  -7.2431228   -5.23307536  -3.20552136
  -1.16490707   0.87856878   0.           0.           0.
   0.           0.        ]
[-11.23657114  -9.25249419  -7.2431228   -5.23307536  -3.20552136
  -1.16490707   0.87856878

In [28]:
z

-95.36443838878465

Now that we have the objective function defined, we need to create a pyPESTO problem.

In [ ]:
def standard_sampling():
    """Creates a pyPESTO problem."""
    objective = pypesto.Objective(fun=negative_log_marginalised_likelihood)
    problem = pypesto.Problem(objective=objective,  # objective function
                              lb=[-5, -5, 0],  # lower bounds
                              ub=[5, 5, np.inf],  # upper bounds
                              x_names=['k1', 'k2', 'scale'],  # parameter names
                              x_scales=['log', 'log', 'lin'])  # parameter scale
    return problem

In [ ]:
### Prior dependent paramters

lamda = 0.01

# create the estimation problem
problem = standard_sampling()

# MCMC chain length
n_samples= 10000

# call the sampler of choice
sampler = sample.AdaptiveMetropolisSampler()

Now we can perform the actual sampling.

In [ ]:
# Define number of runs
runs = 1

save_results = False # for testing just set to False

# Loop over n runs
for n in range(runs):
    # set initial random seed
    np.random.seed(n)
    # perform MCMC sampling
    result = sample.sample(problem, n_samples=n_samples, sampler=sampler,
                           x0=np.array([-1.2741, -0.6160, 0.3684]))
    # calculate effective sample size
    sample.effective_sample_size(result=result)

    # save the results as a pickle object
    if save_results:
        results = [result.sample_result, 'LP_CR_OM']
        with open('Results/Offset_marginalized/' + str(n) + '.pickle','wb') as result_file:
            pickle.dump(results, result_file, n)

There are some built-in visualization functions that one can use.

In [ ]:
# Visualize the parameter trace
ax = visualize.sampling.sampling_parameters_trace(result, use_problem_bounds=False, full_trace=True, size=(12,5))
# Visualize the one-dimensional marginals --> Important!
ax = visualize.sampling_1d_marginals(result, size=(12,5))

In [ ]:
# Visualize the parameter trace
ax = visualize.sampling.sampling_parameters_trace(result, use_problem_bounds=False, full_trace=True, size=(12,5))
# Visualize the one-dimensional marginals --> Important!
ax = visualize.sampling_1d_marginals(result, size=(12,5))